# Scrapy

## **1. Spider 發送初始請求 (Requests) 給 Engine**
- **Spider 是什麼？**
  - **Spider** 是 Scrapy 的「爬蟲邏輯模塊」。你在 Spider 中定義目標網站的爬取規則，例如要爬哪些 URL、如何解析網頁數據，甚至決定是否進一步爬取更多頁面。
  - Spider 是由你自己編寫的，Scrapy 框架會根據你定義的邏輯運行它。

- **初始請求的作用？**
  - Spider 開始時會生成一些初始的 **Requests**，這些 Requests 通常包含網站的起始網址（seed URLs）。
  - 比方說，你可能設定 `start_urls = ['http://example.com']`，這表示爬蟲從這個網址開始工作。

- **初始請求的目標**
  - 請求中可以包含爬蟲的目標，例如：
    - 要爬取的網頁地址。
    - 請求方式（如 GET 或 POST）。
    - 是否需要攜帶 headers（例如模擬瀏覽器）。

---

## **2. Engine 將請求交由 Scheduler 調度**
- **Engine 是什麼？**
  - Engine 是 Scrapy 的「控制中心」，負責協調各模塊之間的數據流動。
  - 它的角色相當於一個指揮官，決定哪些請求該被處理、哪些數據該被交接。

- **Engine 的工作：**
  - 收到 Spider 發送的 Requests。
  - 將 Requests 傳遞給 **Scheduler**（調度器）進行排程。

---

## **3. Scheduler 返回下一個請求給 Engine**
- **Scheduler 是什麼？**
  - Scheduler 是 Scrapy 的「請求管理器」，負責儲存所有待處理的請求，並決定處理的順序。
  - 它會確保請求被有序地執行，並避免不必要的重複請求。

- **Scheduler 的具體功能：**
  1. **儲存請求**：所有來自 Spider 或其他模塊的新 Requests 都會存儲在 Scheduler 中。
  2. **排序邏輯**：Scheduler 可以根據優先級或其他規則，選擇下一個要執行的請求。
  3. **避免重複請求**：Scheduler 通常會檢查是否已經處理過某個請求，從而避免重複爬取相同的網頁。

---

## **4. Engine 經過 Downloader Middlewares 發送請求給 Downloader**
- **Downloader 是什麼？**
  - Downloader 是 Scrapy 的「下載器」，負責執行 HTTP 請求並下載網頁內容。
  - 它可以看作是一個專業的「網頁抓取工具」。

- **Downloader Middleware 是什麼？**
  - **Downloader Middlewares** 是一個可以攔截和修改請求或響應的模塊。
  - 用途包括：
    - 添加自訂 headers（例如模擬用戶代理）。
    - 處理 cookies。
    - 設置代理伺服器。

- **Downloader 的流程：**
  1. Engine 通過 Downloader Middlewares 發送請求給 Downloader。
  2. Downloader 實際執行請求，將網頁數據下載回來。

---

## **5. Downloader 生成 Response**
- **Response 是什麼？**
  - Response 是 Downloader 返回的結果，包含網頁內容及相關的 HTTP 響應數據。
  - 例如，Response 可能包含：
    - 網頁的 HTML 源碼。
    - HTTP 狀態碼（如 200 表示成功，404 表示找不到頁面）。

- **處理完成後：**
  - Downloader 將 Response 返回給 Engine，供後續處理。

---

## **6. Engine 將 Response 傳送給 Spider**
- **Spider 中處理 Response 的邏輯：**
  - Response 傳回給 Spider 後，Spider 會解析網頁內容。
  - 它的工作包括：
    1. **數據提取（item）：**
       - 使用工具（如 XPath 或 CSS Selector）從 HTML 中提取目標數據。例如，從網頁中找到產品名稱或價格。
    2. **生成新請求（Requests）：**
       - 如果需要爬取其他頁面，Spider 可以基於當前頁面的內容生成新的請求。例如，從列表頁進一步爬取每個商品的詳情頁。

- **Spider Middleware 的作用：**
  - **Spider Middlewares** 是在 Spider 接收或發送數據時的中間層，用於攔截或修改數據。

---

## **7. Engine 發送 Item 給 Item Pipelines，處理新 Requests**
- **Item Pipelines 是什麼？**
  - Item Pipelines 是負責處理提取數據（items）的模塊。
  - 功能包括：
    - 清理數據（如去除不必要的字符）。
    - 驗證數據（如檢查欄位是否缺失）。
    - 儲存數據（如寫入資料庫或 CSV 文件）。

- **新的請求（Requests）處理：**
  - Spider 生成的新請求會被 Engine 再次發送回 Scheduler，進一步進行調度。

---

### **總結**
Scrapy 的每個模塊都有特定的職責，它們依序協作完成一次完整的爬取流程。以下是整體架構的運作圖解：

1. **Spider**
   - 定義爬取邏輯，發送初始請求。
2. **Engine**
   - 控制數據流動，協調所有模塊。
3. **Scheduler**
   - 管理請求隊列，選擇下一個請求。
4. **Downloader**
   - 實際下載網頁，返回網頁的 Response。
5. **Spider**
   - 解析網頁數據，生成新的請求。
6. **Item Pipelines**
   - 處理提取的數據（item），完成存儲。

每個步驟中，Scrapy 都使用了「中間件」來允許自定義功能，比如修改請求或處理響應。整個架構的設計高效且靈活，適合複雜的爬取任務。

這樣的架構不僅確保高效執行，還為使用者提供高度的可定制性，例如可以輕鬆調整 Scheduler 的排序邏輯或添加特定的 Downloader Middlewares 來處理反爬措施。

### **6. 實戰項目完整代碼**

使用 Scrapy 從電子商務網站爬取商品數據並存入 SQLite 資料庫。

---

#### **6.1 專案結構**
我們的專案名為 `ecommerce_scraper`，結構如下：
```
ecommerce_scraper/
    scrapy.cfg               # Scrapy 配置文件
    ecommerce_scraper/
        __init__.py
        items.py             # 定義數據結構
        middlewares.py       # 定義中間件
        pipelines.py         # 數據處理與存儲
        settings.py          # 全局設定
        spiders/
            __init__.py
            ecommerce_spider.py  # 爬蟲邏輯
```

---

#### **6.2 定義數據結構（`items.py`）**
在 Scrapy 中，`Item` 類用來定義爬取數據的結構。這讓代碼更具清晰性並便於後續數據處理。

```python
import scrapy

class ProductItem(scrapy.Item):
    # 商品名稱
    name = scrapy.Field()
    # 價格
    price = scrapy.Field()
    # 庫存狀態
    stock = scrapy.Field()
    # 商品網址
    url = scrapy.Field()
```

---

#### **6.3 爬蟲邏輯（`spiders/ecommerce_spider.py`）**
在這裡定義如何從目標網站提取商品信息。

```python
import scrapy
from ecommerce_scraper.items import ProductItem

class EcommerceSpider(scrapy.Spider):
    name = 'ecommerce'  # 爬蟲名稱
    start_urls = ['https://example-ecommerce.com']  # 起始網址

    def parse(self, response):
        # 遍歷每個商品並提取信息
        for product in response.css('div.product'):
            item = ProductItem()
            item['name'] = product.css('h2::text').get()
            item['price'] = product.css('span.price::text').get()
            item['stock'] = product.css('span.stock::text').get()
            item['url'] = product.css('a::attr(href)').get()
            yield item

        # 處理分頁
        next_page = response.css('a.next::attr(href)').get()
        if next_page:
            yield response.follow(next_page, self.parse)
```

**細節剖析：**
1. **`start_urls`**：定義爬取的起始網址。
2. **`parse()`**：是 Scrapy 的默認回調函數，用於解析響應內容。
3. **`response.follow()`**：自動構造下一頁的完整 URL，處理分頁非常方便。

---

#### **6.4 數據處理（`pipelines.py`）**
使用管道將數據存入 SQLite 資料庫。

```python
import sqlite3

class SQLitePipeline:
    def open_spider(self, spider):
        # 建立與 SQLite 資料庫的連接
        self.connection = sqlite3.connect("products.db")
        self.cursor = self.connection.cursor()
        # 建立資料表
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            name TEXT,
            price TEXT,
            stock TEXT,
            url TEXT
        )
        """)
        self.connection.commit()

    def process_item(self, item, spider):
        # 將每個 Item 插入資料庫
        self.cursor.execute("""
        INSERT INTO products (name, price, stock, url) VALUES (?, ?, ?, ?)
        """, (item['name'], item['price'], item['stock'], item['url']))
        self.connection.commit()
        return item

    def close_spider(self, spider):
        # 關閉資料庫連接
        self.connection.close()
```

**細節剖析：**
1. **`open_spider()`**：在爬蟲啟動時打開資料庫連接並初始化資料表。
2. **`process_item()`**：每次處理一個爬取到的 `Item` 並插入資料庫。
3. **`close_spider()`**：在爬蟲結束時關閉資料庫連接，防止資源洩露。

---

#### **6.5 設定（`settings.py`）**
在這裡配置爬蟲的行為，例如延遲、管道啟用等。

```python
# 遵守 robots.txt
ROBOTSTXT_OBEY = True

# 延遲請求
DOWNLOAD_DELAY = 2

# 啟用數據管道
ITEM_PIPELINES = {
    'ecommerce_scraper.pipelines.SQLitePipeline': 300,
}

# 設定 User-Agent
USER_AGENT = 'EcommerceScraper (+http://www.example.com)'
```

**細節剖析：**
1. **`ROBOTSTXT_OBEY`**：設置為 `True` 時會遵守網站的 `robots.txt` 規範。
2. **`DOWNLOAD_DELAY`**：每個請求之間的延遲（以秒為單位），防止對伺服器造成過多壓力。
3. **`ITEM_PIPELINES`**：啟用自定義數據管道，數字代表管道的優先級（數字越小，優先級越高）。

---

#### **6.6 中間件（`middlewares.py`）**
中間件用於攔截和處理請求或回應，例如模擬代理或處理反爬機制。

這裡示例一個自定義 User-Agent 中間件：
```python
class UserAgentMiddleware:
    def process_request(self, request, spider):
        request.headers['User-Agent'] = 'Custom User-Agent String'
```

在 `settings.py` 中啟用：
```python
DOWNLOADER_MIDDLEWARES = {
    'ecommerce_scraper.middlewares.UserAgentMiddleware': 543,
}
```

---

### **7. 運行與測試**

#### **7.1 運行爬蟲**
執行以下命令運行爬蟲：
```bash
scrapy crawl ecommerce
```

#### **7.2 儲存數據到文件**
你可以將數據直接保存為 JSON 或 CSV 文件：
```bash
scrapy crawl ecommerce -o output.json
scrapy crawl ecommerce -o output.csv
```

#### **7.3 檢查資料庫**
打開 SQLite 資料庫，檢查數據是否正確插入：
```bash
sqlite3 products.db
SELECT * FROM products;
```

---

### **8. 最佳實踐與技巧**

1. **善用 Scrapy Shell**：
   - 在爬蟲開發過程中使用 Scrapy Shell 測試選擇器非常重要：
     ```bash
     scrapy shell 'https://example-ecommerce.com'
     ```
   - 測試 CSS 或 XPath 選擇器：
     ```python
     response.css('div.product h2::text').get()
     ```

2. **避免被封鎖**：
   - 設置下載延遲（`DOWNLOAD_DELAY`）。
   - 使用代理服務。
   - 變更 User-Agent 或使用旋轉代理。

3. **處理異常**：
   - 使用 `errback` 處理請求錯誤：
     ```python
     def start_requests(self):
         yield scrapy.Request(
             url="https://example.com",
             callback=self.parse,
             errback=self.handle_error
         )

     def handle_error(self, failure):
         self.logger.error(f'Request failed: {failure}')
     ```

4. **動態網站爬取**：
   - 使用 Scrapy-Selenium 或 Scrapy-Splash 處理 JavaScript 渲染頁面。

5. **數據清理與去重**：
   - 在 Pipeline 中清理不完整或重複的數據。
   - 使用去重管道防止重複插入數據。

---

### **9. 延伸學習與應用**

1. **爬取多層次數據**：
   - 如果一個商品頁面有額外的詳細信息，可以在 `parse` 中生成新的請求，並傳遞當前的數據：
     ```python
     def parse(self, response):
         for product in response.css('div.product'):
             detail_url = product.css('a::attr(href)').get()
             yield response.follow(detail_url, self.parse_detail)

     def parse_detail(self, response):
         # 提取詳細信息
         yield {
             'title': response.css('h1::text').get(),
             'description': response.css('div.description::text').get()
         }
     ```

2. **整合其他技術**：
   - 將爬取的數據與 Pandas 結合，進一步分析或生成圖表。
   - 與 Flask 或 Django 整合，構建 API 以展示爬取的數據。

3. **部署到 Scrapy Cloud**：
   - 使用 Scrapy 的官方服務（Scrapy Cloud）部署並定期執行爬蟲。

---



## **1. `items.py` 語法解釋**

```python
import scrapy

class ProductItem(scrapy.Item):
    name = scrapy.Field()
    price = scrapy.Field()
    stock = scrapy.Field()
    url = scrapy.Field()
```

### **語法詳細解釋**：
1. **`import scrapy`**:
   - 導入 Scrapy 框架的核心模組，包含 `scrapy.Item` 和 `scrapy.Field`。

2. **`class ProductItem(scrapy.Item)`**:
   - 定義一個繼承自 `scrapy.Item` 的數據模型。
   - **`scrapy.Item`** 是 Scrapy 專門用於定義爬取數據結構的基類，作用類似於 Python 的字典，但有更強的數據驗證能力。

3. **`name = scrapy.Field()`**:
   - 定義一個字段（Field），代表爬取數據中的「商品名稱」。
   - **`scrapy.Field()`** 是 Scrapy 提供的一個容器，預設沒有限制，但可以通過擴展添加驗證和預處理邏輯。

4. **`price`, `stock`, `url`**:
   - 與 `name` 類似，分別表示價格、庫存狀態和商品的 URL。

### **為什麼使用 Item？**
相比直接返回 Python 字典，`Item` 提供：
- 更清晰的數據結構。
- 易於在後續 Pipeline 處理數據時進行統一的驗證和格式化。

---

## **2. `spiders/ecommerce_spider.py` 語法解釋**

```python
import scrapy
from ecommerce_scraper.items import ProductItem

class EcommerceSpider(scrapy.Spider):
    name = 'ecommerce'
    start_urls = ['https://example-ecommerce.com']

    def parse(self, response):
        for product in response.css('div.product'):
            item = ProductItem()
            item['name'] = product.css('h2::text').get()
            item['price'] = product.css('span.price::text').get()
            item['stock'] = product.css('span.stock::text').get()
            item['url'] = product.css('a::attr(href)').get()
            yield item

        next_page = response.css('a.next::attr(href)').get()
        if next_page:
            yield response.follow(next_page, self.parse)
```

### **語法詳細解釋**：

#### **`class EcommerceSpider(scrapy.Spider)`**：
1. **`EcommerceSpider`**：
   - 自定義爬蟲類，繼承自 Scrapy 的核心類 `scrapy.Spider`。
   - **`Spider`** 是 Scrapy 的基本單元，負責發送請求和解析回應。

2. **`name = 'ecommerce'`**：
   - 定義爬蟲的名稱，必須是唯一的，用於命令行運行爬蟲：
     ```bash
     scrapy crawl ecommerce
     ```

3. **`start_urls = ['https://example-ecommerce.com']`**：
   - 起始 URL 列表，爬蟲會從這些 URL 開始爬取。

---

#### **`def parse(self, response):`**
1. **`response`**：
   - Scrapy 將網站的 HTTP 回應封裝成一個 `Response` 對象。
   - **主要屬性**：
     - **`response.body`**：返回字節數據（HTML 原始內容）。
     - **`response.text`**：返回解碼後的 HTML 文本。
     - **`response.url`**：獲取當前回應的 URL。
     - **`response.css()` 和 `response.xpath()`**：選擇器，用於提取數據。

---

#### **`for product in response.css('div.product'):`**
- **`response.css()`**：
  - CSS 選擇器，類似於網頁設計中選擇元素的方法。
  - 這裡的 `'div.product'` 是一個選擇器，提取所有 `class="product"` 的 `div` 元素。

---

#### **`item = ProductItem()`**：
1. **創建 Item**：
   - 使用 `ProductItem` 類創建一個 Item 實例，表示要爬取的一條數據。

---

#### **`item['name'] = product.css('h2::text').get()`**：
1. **`product.css('h2::text')`**：
   - 選擇器語法，提取 `<h2>` 元素的文本內容。
   - **`::text`** 表示提取純文本。

2. **`.get()`**：
   - 獲取選擇器的第一個匹配結果（如果沒有匹配結果，返回 `None`）。
   - **其他方法**：
     - **`.getall()`**：返回所有匹配的結果。
     - **`.extract_first()`**：與 `.get()` 等價。

---

#### **`yield item`**：
1. **`yield`**：
   - 將提取到的數據傳遞給 Scrapy 的 Pipeline（管道）處理。
   - **為什麼用 `yield`？**
     - Scrapy 使用生成器處理數據流，`yield` 可以節省內存，並且支持異步操作。

---

#### **處理分頁：**

```python
next_page = response.css('a.next::attr(href)').get()
if next_page:
    yield response.follow(next_page, self.parse)
```

1. **`next_page = response.css('a.next::attr(href)').get()`**：
   - 使用選擇器提取下一頁的 URL（`href` 屬性）。
   - **`::attr(href)`**：提取屬性值。

2. **`response.follow(next_page, self.parse)`**：
   - 使用 Scrapy 的 `follow` 方法生成下一頁的請求。
   - **`self.parse`** 是回調函數，表示下一頁的回應將由 `parse` 方法處理。

---

## **3. `pipelines.py` 語法解釋**

```python
import sqlite3

class SQLitePipeline:
    def open_spider(self, spider):
        self.connection = sqlite3.connect("products.db")
        self.cursor = self.connection.cursor()
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS products (
            name TEXT,
            price TEXT,
            stock TEXT,
            url TEXT
        )
        """)
        self.connection.commit()

    def process_item(self, item, spider):
        self.cursor.execute("""
        INSERT INTO products (name, price, stock, url) VALUES (?, ?, ?, ?)
        """, (item['name'], item['price'], item['stock'], item['url']))
        self.connection.commit()
        return item

    def close_spider(self, spider):
        self.connection.close()
```

---

#### **`open_spider(self, spider)`**
- **目的**：
  - 在爬蟲啟動時自動執行，負責初始化 SQLite 資料庫的連接。

1. **`sqlite3.connect("products.db")`**：
   - 與 `products.db` 資料庫建立連接。
   - 如果資料庫文件不存在，會自動創建。

2. **`CREATE TABLE IF NOT EXISTS`**：
   - 創建 `products` 資料表，確保表只創建一次。

3. **`self.connection.commit()`**：
   - 提交資料表的變更（即使沒有數據插入，也要提交）。

---

#### **`process_item(self, item, spider)`**
1. **`self.cursor.execute()`**：
   - 插入數據到資料表。
   - **`?` 占位符**：防止 SQL 注入攻擊。

2. **`return item`**：
   - 必須返回 `item`，讓 Scrapy 知道這個 Item 已處理完成。

---

#### **`close_spider(self, spider)`**
- **目的**：
  - 爬蟲結束時自動關閉資料庫連接，防止資源洩漏。

---

## **4. `settings.py` 語法解釋**

```python
ROBOTSTXT_OBEY = True
DOWNLOAD_DELAY = 2

ITEM_PIPELINES = {
    'ecommerce_scraper.pipelines.SQLitePipeline': 300,
}

USER_AGENT = 'EcommerceScraper (+http://www.example.com)'
```

### **語法解釋**：
1. **`ROBOTSTXT_OBEY = True`**：
   - 是否遵守 `robots.txt` 規範。
   - **`robots.txt` 是什麼？**
     - 一個網站的規範文件，指導爬蟲哪些內容允許爬取。

2. **`DOWNLOAD_DELAY = 2`**：
  

 - 每個請求之間延遲 2 秒，減少伺服器壓力，避免被封鎖。

3. **`ITEM_PIPELINES`**：
   - 啟用數據處理管道，並設置優先級（數字越小優先級越高）。

4. **`USER_AGENT`**：
   - 定義爬蟲的 User-Agent，模擬瀏覽器請求，避免被識別為爬蟲。

---

## **5. `middlewares.py`（中間件）語法解釋**

中間件 (`middlewares.py`) 是 Scrapy 中一個強大的機制，用於攔截和處理請求或回應。下面是完整的代碼與解析：

```python
class UserAgentMiddleware:
    def process_request(self, request, spider):
        request.headers['User-Agent'] = 'Custom User-Agent String'
```

### **語法解析**：

#### **`process_request(self, request, spider)`**
1. **`request`**：
   - 每次發送 HTTP 請求時，Scrapy 都會調用這個方法。
   - 你可以在這裡修改請求，例如添加標頭（`headers`）或設置代理。

2. **`spider`**：
   - 爬蟲對象，表示當前正在運行的爬蟲。
   - 可以根據 `spider.name` 執行特定操作。

3. **`request.headers['User-Agent']`**：
   - 修改請求的 `User-Agent`，讓爬蟲看起來像一個普通的瀏覽器。

---

### **中間件的工作流程**
1. **請求發出之前**：
   - **Downloader Middleware** 可以修改請求的內容（如 URL、標頭等）。
2. **回應到達之後**：
   - **Spider Middleware** 可以修改回應的內容（如解析頁面之前進行清洗）。

---

## **6. Scrapy Shell（測試代碼）語法解釋**

Scrapy Shell 是一個交互式工具，用於調試選擇器和爬取邏輯。

1. **啟動 Shell**：
   ```bash
   scrapy shell 'https://example.com'
   ```

2. **常用語法**：
   ```python
   response.css('div.product')  # 提取所有 class 為 product 的 div
   response.css('h2::text').get()  # 獲取 h2 元素的第一個文本
   response.xpath('//h2/text()').get()  # 使用 XPath 提取 h2 的文本
   ```

3. **為什麼使用 Shell？**
   - 測試選擇器是否正確提取數據。
   - 避免在爬蟲代碼中反覆修改和運行。

---

## **7. 自定義代碼（處理異常和錯誤回調）語法解釋**

在 Scrapy 中，請求可能會失敗，例如網站超時或伺服器返回 500 錯誤。這時我們需要處理異常。

```python
def start_requests(self):
    urls = ['https://example.com/page1', 'https://example.com/page2']
    for url in urls:
        yield scrapy.Request(
            url=url,
            callback=self.parse,
            errback=self.handle_error
        )

def handle_error(self, failure):
    self.logger.error(f'Request failed: {failure}')
```

### **語法詳細解釋**：

#### **`errback=self.handle_error`**
- 當請求失敗時，會調用 `handle_error` 回調函數。

#### **`self.logger.error()`**
- Scrapy 提供的日誌記錄方法，用於輸出錯誤信息。

#### **`failure`**
- 失敗對象，包含錯誤的詳細信息，例如超時、連接錯誤等。

---

## **8. 完整請求與回應處理邏輯的語法補充**

Scrapy 的請求與回應是封裝後的對象，了解其屬性和方法非常重要。

### **8.1 `Request` 對象**
1. **生成請求**：
   ```python
   scrapy.Request(url="https://example.com", callback=self.parse)
   ```

2. **重要屬性**：
   - **`url`**：請求的目標 URL。
   - **`callback`**：處理回應的回調函數。
   - **`meta`**：攜帶附加數據，例如：
     ```python
     scrapy.Request(url="https://example.com", callback=self.parse, meta={'category': 'Books'})
     ```

### **8.2 `Response` 對象**
1. **回應的屬性**：
   - **`response.url`**：返回響應的 URL。
   - **`response.status`**：HTTP 狀態碼（如 200、404）。
   - **`response.body`**：原始的二進制內容。
   - **`response.text`**：解碼後的 HTML 文本。
   - **`response.css()` 和 `response.xpath()`**：選擇器，用於提取數據。

2. **範例**：
   ```python
   def parse(self, response):
       title = response.css('h1::text').get()
       print(f"Page title: {title}")
   ```

---

## **9. 數據導出與保存語法解釋**

Scrapy 支援將數據保存為 JSON、CSV 或 XML 文件。

1. **命令行直接保存**：
   ```bash
   scrapy crawl ecommerce -o output.json
   ```

2. **在代碼中保存數據**：
   - 使用 Python 的內建文件操作：
     ```python
     def parse(self, response):
         data = {
             'name': response.css('h1::text').get(),
             'price': response.css('.price::text').get()
         }
         with open('output.json', 'a') as f:
             f.write(json.dumps(data) + '\n')
     ```

---

### **10. 常見問題與解法補充**

#### **如何處理反爬機制？**
1. **User-Agent 模擬**：
   - 在 `settings.py` 中設置多個 User-Agent，並隨機使用：
     ```python
     from random import choice
     
     class RandomUserAgentMiddleware:
         def process_request(self, request, spider):
             user_agents = [
                 'Mozilla/5.0 (Windows NT 10.0)',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)'
             ]
             request.headers['User-Agent'] = choice(user_agents)
     ```

2. **代理**：
   - 使用免費或付費代理服務避免 IP 被封鎖。

#### **如何加速開發與調試？**
1. **僅抓取前幾頁**：
   在 `parse` 中設置限制：
   ```python
   def parse(self, response):
       if self.crawler.stats.get_value('item_scraped_count', 0) > 100:
           return  # 停止爬取
   ```

---


